In [1]:
import pandas as pd
import json
import collections
import csv
import math
import numpy as np
import unidecode

In [2]:
def replaceActIds(newdataset):
    newdataset["act"] = newdataset["act"].replace(to_replace="prologue", value=0)
    newdataset["act"] = newdataset["act"].replace(to_replace="act1", value=0.01)
    newdataset["act"] = newdataset["act"].replace(to_replace="act2", value=0.02)
    newdataset["act"] = newdataset["act"].replace(to_replace="act3", value=0.03)
    newdataset["act"] = newdataset["act"].replace(to_replace="act4", value=0.04)
    newdataset["act"] = newdataset["act"].replace(to_replace="act5", value=0.05)
    newdataset["act"] = newdataset["act"].replace(to_replace="act6", value=0.06)
    newdataset["act"] = newdataset["act"].replace(to_replace="act7", value=0.07)
    newdataset["act"] = newdataset["act"].replace(to_replace="act8", value=0.08)
    newdataset["act"] = newdataset["act"].replace(to_replace="act9", value=0.09)
    newdataset["act"] = newdataset["act"].replace(to_replace="act10", value=0.10)
    newdataset["act"] = newdataset["act"].replace(to_replace="act11", value=0.11)
    newdataset["act"] = newdataset["act"].replace(to_replace="act12", value=0.12)
    newdataset["act"] = newdataset["act"].replace(to_replace="act13", value=0.13)
    newdataset["act"] = newdataset["act"].replace(to_replace="act14", value=0.14)
    newdataset["act"] = newdataset["act"].replace(to_replace="act15", value=0.15)
    newdataset["act"] = newdataset["act"].replace(to_replace="act16", value=0.16)
    newdataset["act"] = newdataset["act"].replace(to_replace="act17", value=0.17)
    newdataset["act"] = newdataset["act"].replace(to_replace="act18", value=0.18)
    newdataset["act"] = newdataset["act"].replace(to_replace="act19", value=0.19)
    newdataset["act"] = newdataset["act"].replace(to_replace="act20", value=0.20)
    newdataset["act"] = newdataset["act"].replace(to_replace="credits", value=0.21)
    return newdataset

In [3]:
def convertData(dataframe):
    dataframe = dataframe.reset_index(col_level=1)
    df_m = dataframe[dataframe['gender']=="M"]
    df_f = dataframe[dataframe['gender']=="F"]
    df_all=pd.merge(df_m, df_f, on="episode", how="outer")
    df_all = df_all[["episode","wordCount_x", "wordCount_y"]]
    df_all["wordCount_y"] = df_all["wordCount_y"].fillna(0)
    df_all["wordCount_x"] = df_all["wordCount_x"].fillna(0)
    df_all["total"] = df_all["wordCount_x"] + df_all["wordCount_y"]
    df_all["male%"] = df_all["wordCount_x"] / df_all["total"]
    df_all["female%"] = df_all["wordCount_y"] / df_all["total"]
    df_all.columns=["episode", "male","female","total","male%","female%"]
    return df_all

In [23]:
gender_dataset = pd.read_csv("../data/gender.csv")
acts_dataset = pd.read_table("../data/acts.tsv")
episodes_dataset = pd.read_table("../data/episodes.tsv")
dialogue_dataset = pd.read_table("../data/dialogue.tsv")
dialogue_dataset = replaceActIds(dialogue_dataset)
dialogue_dataset["episode-act"]= dialogue_dataset["episode"].apply(float) + dialogue_dataset["act"].apply(float) 
acts_dataset = replaceActIds(acts_dataset)
acts_dataset["episode-act"]= acts_dataset["episode"].apply(float) + acts_dataset["act"].apply(float) 
acts_dataset.to_csv("../data/actInfo.csv", index=False)

In [24]:
dialogue_dataset = dialogue_dataset[~pd.isnull(dialogue_dataset["dialogue"])]
dialogue_dataset["wordCount"] = dialogue_dataset["dialogue"].map(lambda x: len(x.split()))
dialogue_dataset.to_csv("../data/dialoguebyact.csv", index=False)

In [25]:
export_dataset = dialogue_dataset.groupby(["name","episode","act","gender","role"]).sum()["wordCount"].reset_index()
export_dataset.sort_values(["episode","act","name"])


In [26]:
# get % of male dialogue per episode without Ira
episodeData = dialogue_dataset[(dialogue_dataset["name"]!="Ira Glass") & (dialogue_dataset["name"]!="Announcer")].groupby(["episode","gender"]).sum()["wordCount"].reset_index()
df_all = convertData(episodeData)

In [27]:
#merge to create act 1 csv
mergedAct1 = df_all.merge(episodes_dataset, on="episode")
mergedAct1["male%"] = mergedAct1["male%"] * 100
mergedAct1.columns = ["episode","male","female","total","malePercent","femalePercent","month","day","year","title","description","imageurl","url"]
mergedAct1 = mergedAct1[["episode","malePercent","year","title","description","url"]]
mergedAct1.to_csv("../web/src/assets/data/act1.csv", index=False)

In [9]:
# percentage of episodes that have more than 50% male dialogue
len(mergedAct1[mergedAct1["malePercent"] > 50])/float(len(mergedAct1))

0.7041800643086816

In [10]:
menCount = len(gender_dataset[gender_dataset["gender"]=="M"])
womenCount = len(gender_dataset[gender_dataset["gender"]=="F"])
# percent of speakers that are male
menCount/float(menCount + womenCount)

0.6331263910038655

In [11]:
wordCounts = dialogue_dataset[dialogue_dataset["name"] != "Ira Glass"].groupby(["gender"]).sum()["wordCount"].reset_index()
femaleWordCount = wordCounts.iloc[0]["wordCount"]
maleWordCount = wordCounts.iloc[2]["wordCount"]
# percent of words spoken that are male, excluding Ira Glass
maleWordCount/float(maleWordCount + femaleWordCount)

0.63622855280434232

In [12]:
# average number of men and women in an episode withOUT Ira Glass and Announcer
subset = dialogue_dataset[["episode","name","gender"]].drop_duplicates()
subset = subset[(subset["name"] != "Ira Glass") & (subset["name"] != "Announcer")]
men = subset[subset["gender"]=="M"].groupby("episode").count().reset_index()[["episode", "name"]]
women = subset[subset["gender"]=="F"].groupby("episode").count().reset_index()[["episode", "name"]]
menwomencount = men.merge(women, on="episode")
menwomencount.columns = ["episode", "numMen", "numWomen"]
menwomencount["numMen"].mean(), menwomencount["numMen"].median(), menwomencount["numWomen"].mean(), menwomencount["numWomen"].median()

(10.388059701492537, 9.0, 6.1708126036484243, 5.0)

In [13]:
# In an avg episode, how many words does a woman speak vs a man.
dialogue_dataset2 = dialogue_dataset[(dialogue_dataset["name"]!="Ira Glass") & (dialogue_dataset["name"]!="Announcer")]
men = dialogue_dataset2[(dialogue_dataset2["gender"]=="M")].groupby(["name","episode"]).sum()["wordCount"].reset_index()
women = dialogue_dataset2[(dialogue_dataset2["gender"]=="F")].groupby(["name","episode"]).sum()["wordCount"].reset_index()
men["wordCount"].mean(), men["wordCount"].median(), women["wordCount"].mean(), women["wordCount"].median()

(455.02030063536341, 128.0, 451.6072004298764, 117.0)

In [14]:
# How many appearances in episodes does a man vs woman make?
men = dialogue_dataset2[(dialogue_dataset2["gender"]=="M")].groupby(["name", "episode"]).count()["wordCount"].reset_index().groupby(["name"]).count()["episode"].reset_index()
women = dialogue_dataset2[(dialogue_dataset2["gender"]=="F")].groupby(["name", "episode"]).count()["wordCount"].reset_index().groupby(["name"]).count()["episode"].reset_index()
men["episode"].mean(), men["episode"].median(), women["episode"].mean(), women["episode"].median()

(1.5500840739851069, 1.0, 1.6534873389604621, 1.0)

In [41]:
# percentage of subjects that are male
subjects = dialogue_dataset[dialogue_dataset["role"]=="subject"][["name","gender","episode-act"]].drop_duplicates()
maleSubjectCount = len(subjects[subjects["gender"]=="M"])
femaleSubjectCount = len(subjects[subjects["gender"]=="F"])
maleSubjectCount/float(maleSubjectCount + femaleSubjectCount)

0.6450389503895039

In [51]:
# percentage of speakers that are male
speakers = gender_dataset[["name","gender","episode"]].drop_duplicates()
maleSpeakerCount = len(speakers[speakers["gender"]=="M"])
femaleSpeakerCount = len(speakers[speakers["gender"]=="F"])
print maleSpeakerCount, femaleSpeakerCount, maleSpeakerCount/float(maleSpeakerCount + femaleSpeakerCount)


5405 3132 0.633126391004
